# Audubon in Action: Creative Approaches in Data

During [Earth Week Data Jam](https://www.library.upenn.edu/about/exhibits-events/earth-week-data-jam), join Penn Libraries' Research Data and Digital Scholarship team (and friends) as we create, analyze, visualize, and experiment with data related to environmental issues and the natural world. Whether you're interested in learning how to decode professional data visualizations, working with qualitative data, or building your own data set from the ground up, we hope you find something that meets your needs.

In this notebook, we'll create mosaics, Instagram posts, and a sound file for all the birds in the [Birds of Philadelphia](https://www.inaturalist.org/projects/birds-of-philadelphia) iNaturalist dataset. 

* [Import What We Need](#Import-What-We-Need)
* [Load the Data](#Load-the-Data)
* [Build Mosaics](#Build-a-Mosaic)
* [Build Instagram Posts](#Build-an-Instagram-Post)
* [Build a Sound File](#Build-a-Sound-File)
* [Credits](#Credits)

## Import What We Need

In [ ]:
# Download latest FFmpeg static build.  
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

!pip install -r requirements.txt
import pandas as pd
import os
import subprocess
from PIL import Image  
from PIL import ImageFont
from PIL import ImageDraw
from pydub import AudioSegment
import os, random
import pandas as pd
import numpy as np
import os
import re
#from IPython.display import Image
import datetime
from datetime import date


your_name = input("Include your name for credit: ")

## Load the Data

In [ ]:
directory_path = os.getcwd()
pa_birds = pd.read_csv("data/philly_birds_workshop.csv")

## Build Mosaics

In [ ]:
for i in range(0,pa_birds.shape[0]):
    common_name = pa_birds['bird_name'].iloc[i]
    print(common_name)
    target = pa_birds['audubon_img'].iloc[i]
    target = f'{directory_path}/plates/' + target
    images = pa_birds['inaturalist_folder'].iloc[i]
    images = f'{directory_path}/' + images
    output = pa_birds['mosaic_img'].iloc[i]
    output = f'{directory_path}/' + output
    command = "python Mosaic_Creator.py --target " + target + " --images " + images + " --grid 100 100 --output " + output
    subprocess.run([command],shell=True)

## Build Instagram Posts

In [ ]:
# to build display

width = 1080
height = 1080

for idx, row in pa_birds.iterrows():
    first_line = row.common_name
    second_line = row.scientific_name
    third_line = "Observed " + str(row['count']) + " times since June 19, 2020"
    
    obs = row.observed_on # double check that this works
    obs = datetime.datetime.strptime(obs,'%Y-%m-%d')
    obs = obs.strftime("%B %e, %Y")
    obs = re.sub(' +', ' ', obs)
    
    fourth_line = "Last spotted " + str(obs) + " near " + row.place_guess
    fifth_line = "Created by " + your_name + " for Audubon in Action at the Earth Week Data Jam 2022"

    orig_image = f'{directory_path}/' + row.audubon_img
    created_image = f'{directory_path}/' + row.mosaic_img
    output = f'{directory_path}/' + row.display_img

    img  = Image.new(mode = "RGB", size = (width, height), color = (255,255,255))
    img1 = Image.open(orig_image)
    img2 = Image.open(created_image)
    img1 = img1.resize((400,400))
    img2 = img2.resize((400,400))
    img.paste(img1, (0,0))
    img.paste(img2, (680,0))
    
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Regular.ttf", size=12)
    credit = "Courtesy of the John James Audubon Center at Mill Grove,\nMontgomery County Audubon Collection, and Zebra Publishing"
    draw.text((180,450), credit, (0,0,0), font=font, anchor="mm", align="left")
    
    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Bold.ttf", size=52)
    draw.text((540,520), first_line, (0,0,0), font=font, anchor="mm", align="center")

    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Italic.ttf", size=32)
    draw.text((540,580), second_line, (0,0,0), font=font, anchor="mm", align="center")

    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Regular.ttf", size=22)
    draw.text((540,680), third_line, (0,0,0), font=font, anchor="mm", align="center")

    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Regular.ttf", size=22)
    draw.text((540,720), fourth_line, (0,0,0), font=font, anchor="mm", align="center")

    font = ImageFont.truetype(f"{directory_path}/fonts/Roboto/Roboto-Regular.ttf", size=12)
    draw.text((540,1000), fifth_line, (0,0,0), font=font, anchor="mm", align="center")

    img.save(output)

## Build a Sound File

Now we'll build a unified sound file of all the birds represented in the project. 

In [ ]:
inaturalist_data = pd.read_csv("data/observations-218728.csv")
list_of_licenses = ['CC-BY-NC', 'CC-BY-NC-SA', 'CC-BY', 'CC0', 'CC-BY-SA', 'CC-BY-NC-ND']
inaturalist_data = inaturalist_data.loc[inaturalist_data['license'].isin(list_of_licenses)].reset_index() 
birds_to_include = pa_birds['common_name'].to_list()
inaturalist_data = inaturalist_data[inaturalist_data['common_name'].isin(birds_to_include)]
inaturalist_data = inaturalist_data[inaturalist_data['observed_on'].str.startswith('2021')].reset_index()
dates = inaturalist_data['observed_on'].to_list()
start_date = datetime.datetime.strptime("2021-1-1",'%Y-%m-%d')
days_dates = {}
for day in dates: 
    end_date = datetime.datetime.strptime(day, '%Y-%m-%d')
    delta = end_date - start_date
    days_dates[delta.days] = day

In [ ]:
# to display sound
from pydub.playback import play
res = int(list(days_dates.keys())[0])
all_sounds = AudioSegment.silent(duration=1)
all_sounds.duration_seconds == int(delta.days)

for i in range(res,int(delta.days)):
    print(i)
    if i in days_dates.keys():
        print("yes")
        value = days_dates[i]
        spotted_birds = inaturalist_data[inaturalist_data['observed_on']==value].reset_index(drop=True)
        print(spotted_birds.shape[0])
        common_name = spotted_birds['common_name'].loc[spotted_birds.index[0]]
        print(common_name)
        bird_name = common_name.lower()
        bird_name = '-'.join(bird_name.split())
        sounds_folder = f'{directory_path}/sounds/' + bird_name
        sound_file = random.choice(os.listdir(sounds_folder))
        sound_type = sound_file.rsplit('.')[1]
        sound_file = sounds_folder + "/" + sound_file
        print(sound_file)
        sound = AudioSegment.from_file(sound_file, sound_type)
        sound = sound[1000:2000]
        combined = sound + 10
        for idx, row in spotted_birds[1:].iterrows(): 
            common_name = spotted_birds['common_name'].loc[spotted_birds.index[idx]]
            bird_name = common_name.lower()
            bird_name = '-'.join(bird_name.split())
            sounds_folder = f'{directory_path}/sounds/' + bird_name
            sound_file = random.choice(os.listdir(sounds_folder))
            sound_type = sound_file.rsplit('.')[1]
            sound_file = sounds_folder + "/" + sound_file
            sound = AudioSegment.from_file(sound_file, sound_type)
            sound = sound[1000:2000]
            sound = sound + 15
            combined.overlay(sound)
            #play(combined)
    else:
        print("no")
        combined = AudioSegment.silent(duration=1000)
    all_sounds = all_sounds + combined
all_sounds.export("sounds/2021-all-output.mp3", format="mp3")
print("sound created!")

## Credits

Created by [Emily Esten](https://www.library.upenn.edu/people/staff/emily-esten) for Earth Week Data Jam 2022.

The Center for Research Data and Digital Scholarship facilitates data-driven and data-literate research and scholarship across the disciplines in order to fster informed and ethical data communites at Penn. Interested in data, computational research, digital humanities, or open and public scholarship? Find us on the [Penn Libraries website](https://www.library.upenn.edu/help-with/research-data-digital-scholarship). 